In [ ]:
import pandas as pd
import os
import json
import asyncio
import re

from openai import OpenAI
from openai import AsyncOpenAI

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = None

OPENAI_KEY = ""

In [ ]:
!pip install yake

In [ ]:
from collections import Counter
from bs4 import BeautifulSoup
import re
import yake
import nltk

# Download stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

def parse_html(html):
    if not html or html == '[]':
        return ""

    # Detecta string de bytes no formato hexadecimal
    if isinstance(html, str):
        hex_pattern = re.fullmatch(r'([0-9A-Fa-f]{2}(\s+|$))+', html.strip())
        if hex_pattern:
            try:
                byte_list = [int(x, 16) for x in html.strip().split()]
                html = bytes(byte_list)
            except Exception:
                return ""

    # Decodifica bytes para string UTF-8
    if isinstance(html, bytes):
        try:
            html = html.decode('utf-8', erros='raise')
        except Exception:
            return ""
    
    # Faz o parsing com BeautifulSoup
    try:
        soup = BeautifulSoup(html, 'html.parser')
        # Remove tags desnecessárias
        for tag in soup(["script", "style", "meta", "link", "noscript"]):
            tag.decompose()

        # Extrai o texto e limpa
        text = soup.get_text(separator=' ', strip=True)
        text = re.sub(r'\b\d+(\.\d+)?\b', '', text)  # Removes numerical values (e.g., 123, 123.45)
        text = re.sub(r'\b\d{1,2}h(\d{2})?\b', '', text)  # Remove hours (e.g., 12h30, 3h)
        return text.strip()
    except Exception:
        return "failed to parse HTML"

def extract_yake_keywords(text):
    if not text:
        return []
    try:
        kw_extractor = yake.KeywordExtractor(
            lan="pt",
            n=1,
            dedupLim=0.9,
            top=100,
            features=None
        )
        keywords = kw_extractor.extract_keywords(text)
        return [kw[0].lower() for kw in keywords]
    except:
        return []

def extract_common_words(text):
    if not text:
        return []
    try:
        words = re.findall(r'\b\w+\b', text.lower())
        filtered_words = [word for word in words if word not in stopwords and len(word) > 2]
        word_counts = Counter(filtered_words)
        return [word[0].lower() for word in word_counts.most_common(100)]
    except:
        return []

def concatenate_keywords(html):
    text = parse_html(html)

    yake_keywords = extract_yake_keywords(text)
    common_words = extract_common_words(text)
    combined = []
    max_length = max(len(yake_keywords), len(common_words))
    for i in range(max_length):
        if i < len(yake_keywords) and yake_keywords[i] not in combined:
            combined.append(yake_keywords[i])
        if i < len(common_words) and common_words[i] not in combined:
            combined.append(common_words[i])
    return combined[:150]

# Labeling

In [ ]:
df = pd.read_parquet("../data/novo_dado_iugu_enriquecimento.parquet", engine="pyarrow").reset_index(drop=True)
df = df[["url", "host", "html"]]
df.head(50)

In [ ]:
labels_df = pd.read_csv("../data/IUGU Label - Base para enriquecimento.csv").reset_index(drop=True)
labels_df = labels_df.rename(columns={"Site": "host"})
labels_df["host"] = labels_df["host"].str.replace("https://", "")
labels_df["host"] = labels_df["host"].str.replace(".br/", ".br")
labels_df["host"] = labels_df["host"].str.replace(".com/", ".com")
labels_df["host"] = labels_df["host"].str.replace(".net/", ".net")
labels_df["host"] = labels_df["host"].str.replace("www.", "")
labels_df["cnpj"] = labels_df["cnpj"].astype(str)
labels_df.head()

In [ ]:
merged_df = labels_df.merge(df, on="host", how="left")
merged_df.head(50)

In [ ]:
df = merged_df.copy()
df.shape

In [ ]:
df

In [ ]:
df["yake_keywords"] = df["html"].apply(concatenate_keywords)
df["yake_keywords"]

In [ ]:
df.isnull().sum()

In [ ]:
BATCH_SIZE = 50
TEST_LEN = 250
BACKUP_PATH = "../data/sample_classified.csv"

class Classifier:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    @staticmethod
    def _safe_get(value):
        return value if value not in [None, ""] else "Não informado"
    
    @staticmethod
    def _output_parser(content):
        match = re.search(r'\{.*\}', content, re.DOTALL)

        if match:
            json_string = match.group(0)
            data = json.loads(json_string)
            data = {k: data.get(k) for k in ['CNPJ', 'Segmento', 'Subsegmento', 'Nicho Tech']}
            return data
        else:
            print("No JSON found")

    def prompt(self, row: pd.Series):
        return f""" 
        Analise as informações abaixo sobre uma empresa e classifique-a em um dos seguintes segmentos, subsegmentos e nichos segundo as tabelas :

        |  Segmento  	|            Subsegmento            	|
        |:----------:	|:---------------------------------:	|
        |    Agro    	|            Agricultura            	|
        |    Agro    	|            Outro (Agro)           	|
        |    Agro    	|              Pecuária             	|
        |     Bet    	|                Bet                	|
        |  Comércio  	|              Atacado              	|
        |  Comércio  	|          Outro (Comércio)         	|
        |  Comércio  	|               Varejo              	|
        |  Educação  	|          Educação básica          	|
        |  Educação  	|        Educação Continuada        	|
        |  Educação  	|         Educação Superior         	|
        |  Educação  	|          Outro (Educação)         	|
        | Financeiro 	|               Banco               	|
        | Financeiro 	|              Cobrança             	|
        | Financeiro 	|             Corretora             	|
        | Financeiro 	|     Instituição de pagamentos     	|
        | Financeiro 	|        Mercado de capitais        	|
        | Financeiro 	|         Outro (Financeiro)        	|
        | Financeiro 	|             Seguradora            	|
        |  Indústria 	|            Alimentício            	|
        |  Indústria 	|            Eletricidade           	|
        |  Indústria 	|             Eletrônico            	|
        |  Indústria 	|             Hospitalar            	|
        |  Indústria 	|             Madeireira            	|
        |  Indústria 	|            Metalúrgica            	|
        |  Indústria 	|             Mineradora            	|
        |  Indústria 	|         Outro (Indústria)         	|
        |  Indústria 	|            Petrolífera            	|
        |  Indústria 	|              Químico              	|
        |    Outro   	|           Outro (Outro)           	|
        |    Saas    	|            Marketplace            	|
        |    Saas    	|            Outro (Saas)           	|
        |    Saas    	|             Plataforma            	|
        |    Saas    	|              Software             	|
        |    Saúde   	| Centro clínico ou rede de clínica 	|
        |    Saúde   	|         Cooperativa médica        	|
        |    Saúde   	|              Hospital             	|
        |    Saúde   	|    Operadora de planos de saúde   	|
        |    Saúde   	|           Outro (Saúde)           	|
        |  Serviços  	|              Agência              	|
        |  Serviços  	|            Alimentação            	|
        |  Serviços  	|            Consultoria            	|
        |  Serviços  	|              Estatal              	|
        |  Serviços  	|             Hotelaria             	|
        |  Serviços  	|            Imobiliária            	|
        |  Serviços  	|              Jurídico             	|
        |  Serviços  	|          Outro (Serviços)         	|
        |  Serviços  	|                T.I                	|
        |  Serviços  	|          Telecomunicação          	|

        | Segmento 	|  Nicho Tech 	|
        |:--------:	|:-----------:	|
        |   Saas   	|   Bettech   	|
        |   Saas   	|    Edtech   	|
        |   Saas   	|    Adtech   	|
        |   Saas   	|   Agrotech  	|
        |   Saas   	|   Biotech   	|
        |   Saas   	| Construtech 	|
        |   Saas   	|  Energytech 	|
        |   Saas   	|   Fintech   	|
        |   Saas   	|   Foodtech  	|
        |   Saas   	|   Govtech   	|
        |   Saas   	|    Hrtech   	|
        |   Saas   	|   Indtech   	|
        |   Saas   	|  Insurtech  	|
        |   Saas   	|  Legaltech  	|
        |   Saas   	|   Proptech  	|
        |   Saas   	|  Retailtech 	|
        |   Saas   	|  Sporttech  	|
        |   Saas   	|  Healthtech 	|
        |   Saas   	|  Outra tech 	|

        Dados da empresa:

        CNPJ: {self._safe_get(row.get('cnpj'))}
        Nome: {self._safe_get(row.get('nome'))}
        CNAE: {self._safe_get(row.get('cnae'))}
        Domínio: {self._safe_get(row.get('url'))}
        Tokens: {self._safe_get(row.get('yake_keywords'))}

        Regras de classificação: 
            1) O SaaS deve ser o primeiro a ser classificado em relação aos outros segmentos, caso haja ambiguidade entre o segmento Saas e algum outro segmento. Vendo que uma determinada empresa não se enquadra como Saas, deve-se classifica-la dentro dos outros segmentos possíveis. A partir disso, classificar o subsegmento seguindo a tabela de referencia. Para as empresas classificadas como Saas, deve-se também, classificar o Nicho delas, seguindo a tabela de referencia de Segmento e Nicho Tech.
            2) Na classificação, analise o html do site, e verifique se a partir dessas informações, é possível classificar o segmento e subsegmento da empresa. Caso haja alguma dúvida, utilize o CNAE como fator de desempate na classificação.
            3) Se o site não tiver domínio e tokens, classifique-o de acordo com o CNAE e o nome. Não retorne uma saída vazia, o importante é classificar. Se não se encaixar em nenhum dos segmentos, colocar "Não listado" para todos os campos.
            
        Definições:
            - SaaS, ou Software as a Service (Software como Serviço), é um modelo de software onde o fornecedor hospeda e mantém aplicações na nuvem, disponibilizando-as aos clientes por meio da internet, geralmente através de uma assinatura. Os clientes não precisam instalar ou gerenciar o software, acessando-o via navegador e pagando conforme o uso.
            - Bettech: Bettech: Tecnologia voltada para o setor de apostas, tanto esportivas quanto de cassino, com foco em melhorar a experiência do usuário, segurança e análise de dados. Exemplos: Bet365, Betfair, Sportingbet.
            - Edtech: Tecnologia aplicada à educação para facilitar o ensino e a aprendizagem por meio de plataformas digitais, apps e inteligência artificial. Exemplos: Duolingo, Khan Academy, Coursera.
            - Adtech: Ferramentas e plataformas tecnológicas que otimizam campanhas de publicidade digital, como segmentação, análise de dados e automação. Exemplos: Google Ads, The Trade Desk, Taboola.
            - Agrotech:	Soluções tecnológicas aplicadas ao agronegócio para melhorar produtividade, sustentabilidade e gestão de recursos. Exemplos: Solinftec, Agrotools, John Deere (com tecnologia de agricultura de precisão).
            - Biotech: Uso de processos biológicos e organismos vivos para desenvolver produtos e soluções, especialmente em saúde, genética e meio ambiente. Exemplos: Moderna, Biogen, Amgen.
            - Construtech: Construtech: Inovações tecnológicas na construção civil, como software de gestão de obras, impressão 3D de estruturas e automação de processos. Exemplos: Construct App, 123Projetei, Ambar.
            - Energytech: Tecnologias que otimizam a geração, distribuição, armazenamento ou uso de energia, especialmente renovável. Exemplos: Tesla Energy, Sonnen, Enphase.
            - Fintech: Soluções tecnológicas no setor financeiro, como bancos digitais, pagamentos online, investimentos e seguros. Exemplos: Nubank, PicPay, Revolut.
            - Foodtech:	Inovação tecnológica na produção, distribuição e consumo de alimentos, incluindo delivery, alimentos alternativos e rastreabilidade. Exemplos: iFood, Impossible Foods, NotCo.
            - Govtech: Tecnologia voltada para melhorar os serviços públicos e a gestão governamental com mais eficiência e transparência. Exemplos: Colab, Portal da Transparência, Brasil Cidadão.
            - Hrtech: Soluções tecnológicas aplicadas à gestão de recursos humanos, como recrutamento, folha de pagamento e engajamento de colaboradores. Exemplos: Gupy, Kenoby, Revelo.
            - Indtech: Inovações tecnológicas no setor industrial, incluindo automação, IoT, robótica e monitoramento de processos. Exemplos: Siemens (com soluções industriais), Rockwell Automation, Embraer (com Indústria 4.0).
            - Insurtech: Tecnologia aplicada ao setor de seguros, oferecendo produtos personalizados, automação de processos e melhoria na experiência do cliente. Exemplos: Youse, Pier, Lemonade.
            - Legaltech: Uso da tecnologia para tornar os serviços jurídicos mais acessíveis, rápidos e eficientes. Exemplos: JusBrasil, LegalOne, Docket.
            - Proptech: Soluções tecnológicas no setor imobiliário, como compra, aluguel e gestão de imóveis online. Exemplos: QuintoAndar, Loft, Zillow.
            - Retailtech: Inovações no setor varejista, como e-commerce, automação de lojas, personalização e gestão de estoque. Exemplos: VTEX, Magalu, Shopify.
            - Sporttech: Tecnologia aplicada ao esporte para melhorar desempenho de atletas, experiência do público e gestão de eventos esportivos. Strava, Catapult Sports, OneFootball.
            - Healthtech: Soluções tecnológicas voltadas para saúde e bem-estar, como telemedicina, monitoramento remoto e prontuário eletrônico. Exemplos: Dr. Consulta, Teladoc Health, Vitalk.
            - Outra tech: Qualquer plataforma que não se encaixe nas anteriores.

        Exemplos de SaaS:
            - Serviços de e-mail (como Google Workspace e Microsoft Office 365).
            - Plataformas de CRM (como Salesforce e Zendesk).
            - Ferramentas de produtividade e colaboração (como Slack, Trello e Asana).
            - Plataformas de streaming (como Netflix e Spotify).
            - Sistemas de ERP (como Oracle NetSuite e Conta Azul). 

        Responda, com base exclusivamente na estrutura da tabela apresentada, o CNPJ da empresa, o segmento, o subsegmento e o nicho tech apenas se o segmento for Saas (se não for deixar em branco), em formato JSON com as chaves 'CNPJ', 'Segmento', 'Subsegmento' e 'Nicho Tech', respectivamente.

        Resposta:
        """
    
    def classify(self, row: pd.Series):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {"role": "user", "content": self.prompt(row)}
            ],
            model="gpt-4o-mini",
            temperature=0,
            max_tokens=100,
        )

        data = self._output_parser(chat_completion.choices[0].message.content)
        return data
    
        
    async def parallel_classify(self, X: pd.DataFrame, batch_size: int=10, sleep=0.0):
        """Classifies data in batches asynchronously and yields it"""
        
        async_client = AsyncOpenAI(api_key=self.client.api_key)
        all_results = {}

        for batch_start in range(0, len(X), batch_size):
            batch_end = min(batch_start + batch_size, len(X))
            batch = X.iloc[batch_start:batch_end]

            tasks = []
            batch_indices = []

            for idx, row in batch.iterrows():
                prompt = [{'role': 'user', 'content': self.prompt(row)}]
                task = async_client.chat.completions.create(
                    messages=prompt,
                    model="gpt-4o-mini",
                    temperature=0,
                    max_tokens=100,
                )
                tasks.append(task)
                batch_indices.append(idx)

            # Execute batch asynchronously
            batch_responses = await asyncio.gather(*tasks)

            batch_results = {}
            for idx, response in zip(batch_indices, batch_responses):
                print(response.choices[0].message.content)
                data = self._output_parser(response.choices[0].message.content)

                batch_results[idx] = data
                all_results[idx] = data

            # Yield results per batch
            yield pd.DataFrame.from_dict(batch_results, orient='index')

            # Delay between batches
            await asyncio.sleep(sleep)

async def annotate(df, classifier, batch_size=50):
    """Yields batches of annotated data, including original DataFrame columns."""
    async for batch in classifier.parallel_classify(df, batch_size=batch_size):
        # Create a DataFrame for the batch with original df index
        annotated_batch = pd.DataFrame(batch, index=df.loc[batch.index].index)

        yield annotated_batch

def append_if_exists(df: pd.DataFrame, path: str):
    """Appends `df` to an existing file if it exists, otherwise creates a new file.
    
    Supports both CSV and Parquet formats. If a Parquet file is stored as a folder, it loads and appends correctly.
    
    Args:
        df (pd.DataFrame): The DataFrame to append.
        path (str): The file path, should end in .csv or .parquet.
    """
    existing_df = load_file_if_exists(path)

    if not existing_df is None:
        df = pd.concat([existing_df, df], ignore_index=True)

    # Save the DataFrame in the correct format
    if path.endswith(".csv"):
        print('saved at', path)
        df.to_csv(path, index=False, sep=';')
    elif path.endswith(".parquet"):
        df.to_parquet(path, index=False)  # Will save as a folder if partitioning is used
    else:
        raise ValueError("Unsupported file format. Use .csv or .parquet")
    
    return df

def load_file_if_exists(path: str):
    """Checks and open a file if it exists.
    
    Supports both CSV and Parquet formats. If a Parquet file is stored as a folder, it loads and appends correctly.
    
    Args:
        path (str): The file path, should end in .csv or .parquet.
    """
    file_exists = os.path.exists(path) or os.path.isdir(path)  # Check if it's a folder (Parquet case)

    if file_exists:
        if path.endswith(".csv"):
            existing_df = pd.read_csv(path, sep=';')
        elif path.endswith(".parquet"):
            existing_df = pd.read_parquet(path)  # Reads the folder as a Parquet dataset
        else:
            raise ValueError("Unsupported file format. Use .csv or .parquet")
        
        return existing_df
    
    return None

In [ ]:
classifier = Classifier(api_key=OPENAI_KEY)
samples = []
# df = df.drop(columns=["Segmento iugu", "Nicho Tech"])
df_copy = df.copy()
existing_samples = load_file_if_exists(BACKUP_PATH)

# if not existing_samples is None:
#     # excluding the samples that already were processed
#     unique_domains = existing_samples["cnpj"].unique().tolist()
#     df = df[~df["cnpj"].isin(unique_domains)].reset_index(drop=True)

async for batch in annotate(df, classifier, batch_size=BATCH_SIZE):
    samples.append(batch)
    append_if_exists(batch, BACKUP_PATH)  # Append in batches
    print(f'{min(BATCH_SIZE, len(batch))} classified samples were added to {BACKUP_PATH}')

labeled_df = pd.concat(samples, axis=0)
labeled_df = labeled_df.rename(columns={"CNPJ": "cnpj"})
labeled_df_copy = labeled_df.copy()
# labeled_df = labeled_df[labeled_df["Segmento"] != "Não listado"].reset_index(drop=True)
# labeled_df = labeled_df[labeled_df["Nicho Tech"] != "Não listado"].reset_index(drop=True)

if df.shape[0] != df_copy.shape[0]:
    labeled_df = pd.merge(left=df_copy, right=labeled_df, how="inner", on="cnpj")
else:
    labeled_df = pd.merge(left=df, right=labeled_df, how="inner", on="cnpj")

labeled_df

In [ ]:
labeled_df_copy

In [ ]:
# labeled_df = labeled_df[["url", "host", "cnpj", "raiz_cnpj", "nome", "cnae", "Segmento", "Nicho Tech"]]
# labeled_df = labeled_df.rename(columns={"Nicho Tech_y": "Nicho Tech"})
labeled_df.columns.to_list()

In [ ]:
# labeled_df = labeled_df[
#     ['cnpj',
#  'nome',
#  'Situacao cadastral',
#  'CNAE',
#  'host',
#  'url',
#  'Segmento',
#  'Subsegmento',
#  'Nicho Tech']
# ]

labeled_df.to_csv("../data/iugu_enrichment_chatgpt.csv", index=False)

In [ ]:
# labeled_df